In [ ]:
setwd("../../../../")

In [ ]:
RhpcBLASctl::blas_set_num_threads(1)
library(Seurat)
library(dplyr)
#library(tidyverse)
library(here)
#library(readxl)
#library(Matrix)
library(IMSBSeuratHelpers)
library(ggplot2)
#library(sctransform)
library(future)
library(data.table)
library(xlsx)
#plan("multiprocess", workers = 20)
options(future.globals.maxSize = 20 * 1000 * 1024^2)#20GB
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)

In [ ]:
#library(devtools)
#install_github("immunogenomics/harmony")

In [ ]:
library(harmony)

In [ ]:
samples_ID="Harmony_Ustekinumab_4PK"

In [ ]:
ob.list<-list()
addlist <- 
c(
'P069',
'P094',
'P140',
'P162')

samplelist <- 
c(
'P069_K',
'P094_fullCITE_K',
'P140K_filtered',
'P162K_filtered'
)


#hvg_names<-c()
for (i in 1:length(addlist)) {
    patient_ID <- addlist[[i]]
    sample_ID <- samplelist[[i]]
    ob.list[[i]]<-readRDS(here("Prefiltering_objects",paste0(sample_ID,".rds")))
    ob.list[[i]]<-RenameCells(ob.list[[i]],
                              add.cell.id = paste0(patient_ID,"K"))
    #ob.list[[i]]@meta.data$patient<-ob.list[[i]]@meta.data$orig.ident
    ob.list[[i]]@meta.data$patient<-patient_ID
    ob.list[[i]]@meta.data$sample<-paste0(ob.list[[i]]@meta.data$patient,'K')
    ob.list[[i]]@meta.data$case<-substr(ob.list[[i]]@meta.data$sample[1], 1,1)
    ob.list[[i]]@meta.data$tissue<-substr(ob.list[[i]]@meta.data$sample[1], 5,5)
        DefaultAssay(object = ob.list[[i]]) <- "RNA"
    ob.list[[i]] <- NormalizeData(object = ob.list[[i]],
                     normalization.method = "LogNormalize",
                     scale.factor = 10000)
    ob.list[[i]] <- FindVariableFeatures(object = ob.list[[i]], 
        selection.method = "vst", nfeatures = 2000, verbose = FALSE)
}
ob.list

In [ ]:
seurat.combined<-merge(ob.list[[1]],ob.list[-1]#,
                       #add.cell.ids = addlist
                      )
head(seurat.combined@meta.data)
tail(seurat.combined@meta.data)

In [ ]:
Idents(seurat.combined)<-'patient'
levels(seurat.combined)

In [ ]:
# nCells in total 
paste0(length(ob.list)," samples,", nrow(seurat.combined@meta.data)," cells in total")
sample_nCell<-as.data.frame(table(seurat.combined@meta.data$sample))
colnames(sample_nCell)<- c("sample","nCells")
rbind(sample_nCell, data.frame(sample='Total', nCells = sum(sample_nCell$nCells)))

In [ ]:
seu<-seurat.combined
seu

In [ ]:
seu <- count_fractions(seu,species = "human")

In [ ]:
head(seu@meta.data)
tail(seu@meta.data)

In [ ]:
seu <- ScaleData(seu, features = rownames(seu))

In [ ]:
seu <- FindVariableFeatures(object = seu, 
                            selection.method = "vst",
                            nfeatures = 2000)

In [ ]:
seu <- RunPCA(object = seu, 
              features = VariableFeatures(object = seu),
              verbose = T)

In [ ]:
seu <- seu %>% 
    RunHarmony("patient", dims.use = 1:30,
               plot_convergence = TRUE)

In [ ]:
#seu <- RunTSNE(object = seu,reduction = 'harmony', dims = 1:30)
seu <- RunUMAP(object = seu,reduction = 'harmony', dims = 1:30)

In [ ]:
#DimPlot(object = seu, reduction = 'tsne',label = F, 
#        pt.size = 0.1)+ theme(aspect.ratio=1)
DimPlot(object = seu, reduction = 'umap',label = F, 
        pt.size = 0.1)+ theme(aspect.ratio=1)

In [ ]:
sample_names<-unique(seu@meta.data$patient)
for (i in c(1:length(sample_names))){
    highlightcells <- list(WhichCells(object = seu, 
                                      idents = sample_names[i]))
    names(highlightcells) <- sample_names[i]
    print(
        DimPlot(object = seu, pt.size = 1,reduction = 'umap',
                label = F, 
         cells.highlight = highlightcells,
         cols.highlight = scales::hue_pal()(length(sample_names))[i],
         sizes.highlight = 1,
         #plot.title=sample_names[i]#,
                )+ theme(aspect.ratio=1)
         ) 
    
}

In [ ]:
marker_qc<-c("nFeature_RNA","nCount_RNA","frac.mito","frac.ribo","CD3_count")
for (n in marker_qc){
    print(VlnPlot(object = seu, features = n,
                  group.by = "patient",
                  #split.by = "cellgroup",
                  #x.lab.rot=T, 
                  #size.x.use = 5,
                  pt.size = 0
                       )#+ NoLegend()
    )

    print(FeaturePlot(object = seu,features = n,
                  cols = c("grey", "blue"),
                      order = T,
                      reduction = "umap",
                pt.size=0.1
                       ))     
    }

In [ ]:
seu <- FindNeighbors(object = seu,reduction = 'harmony', dims = 1:30)
#!!! Essential step to set the reduction to 'harmony', otherwise the default is PCA and will give big difference of clusters and UMAP
#seu <- FindNeighbors(object = seu, dims = 1:30)

In [ ]:
seu <- FindClusters(object = seu, resolution = 0.1)
table(Idents(seu))
#DimPlot(object = seu, reduction = 'tsne',label = TRUE, 
#        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)
DimPlot(object = seu, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
seu <- FindClusters(object = seu, resolution = 0.2)
table(Idents(seu))
#DimPlot(object = seu, reduction = 'tsne',label = TRUE, 
#        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)
DimPlot(object = seu, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
seu <- FindClusters(object = seu, resolution = 0.3)
table(Idents(seu))
#DimPlot(object = seu, reduction = 'tsne',label = TRUE, 
#        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)
DimPlot(object = seu, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
seu <- FindClusters(object = seu, resolution = 0.4)
table(Idents(seu))
#DimPlot(object = seu, reduction = 'tsne',label = TRUE, 
#        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)
DimPlot(object = seu, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
seu <- FindClusters(object = seu, resolution = 0.5)
table(Idents(seu))
#DimPlot(object = seu, reduction = 'tsne',label = TRUE, 
#        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)
DimPlot(object = seu, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
seu <- FindClusters(object = seu, resolution = 0.6)
table(Idents(seu))
#DimPlot(object = seu, reduction = 'tsne',label = TRUE, 
#        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)
DimPlot(object = seu, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
seu <- FindClusters(object = seu, resolution = 0.6)
table(Idents(seu))
#DimPlot(object = seu, reduction = 'tsne',label = TRUE, 
#        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)
DimPlot(object = seu, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive ones
seu.markers <- FindAllMarkers(object = seu, only.pos = TRUE, 
                              min.pct = 0.25, logfc.threshold = 0.5)
#logfc.threshold=0.25 (default) instead of old version thresh.use=0.25
head(seu.markers)
dim(seu.markers)

In [ ]:
top3 <- seu.markers %>% group_by(cluster) %>% top_n(3, avg_log2FC)
top5 <- seu.markers %>% group_by(cluster) %>% top_n(5, avg_log2FC)
top10 <- seu.markers %>% group_by(cluster) %>% top_n(10, avg_log2FC)

In [ ]:
top10

In [ ]:
plt1<-DotPlot(seu, features = unique(top5$gene),
              dot.scale = 2
              #scale.by = "size"
             ) + coord_flip()+
theme(#strip.background = element_blank(),
    axis.title = element_blank(),
    axis.text.x = element_text(size = 8,angle = 45,hjust = 1),
    axis.text.y = element_text(size = 8),
     legend.position = "right",
     #legend.spacing = unit(0, "mm"),
     legend.direction = "vertical",
        legend.text = element_text(size=5),
        legend.key.width = unit(2, "mm"),
        legend.key.height = unit(2, "mm"),
        legend.box.spacing = unit(1, "mm"),
        legend.margin = margin(2),
        legend.title = element_text(size = 7,angle = 90)
)
plt1

In [ ]:
DefaultAssay(seu) <- "CITE"
seu<- NormalizeData(object = seu,
                     normalization.method = "LogNormalize",
                     scale.factor = 10000)

In [ ]:
#FeatureScatter(seu, feature1 = "cite_CD4", feature2 = "cite_CD8")
#FeatureScatter(seu, feature1 = "cite_CD4", feature2 = "cite_CD8",
#              slot = "counts")

In [ ]:
CITE_gene<-
c(
"cite_CD45",
"cite_CD3",
"cite_CD4",
"cite_CD8",
"cite_TCRab",
"cite_TCRgd",
"cite_TCRVa7.2",
"cite_TCRVa24Ja18",
"cite_CD45RA",
"cite_CD45RO",
"cite_PDL1",
"cite_CD154",
"cite_CD56",
"cite_CD14",
"cite_CD19",
"cite_HLAABC",
"cite_CD7",
"cite_CD49f",
"cite_CD25",
"cite_TIGIT",
"cite_CTLA4",
"cite_CD31",
"cite_CD44",
"cite_PDPN",
"cite_CD103",
"cite_CD69",
"cite_CD62L",
"cite_CD161",
"cite_LAG3",
"cite_CD27",
"cite_LAMP1",
"cite_HLADR",
"cite_NKG2D",
"cite_CD57",
"cite_KLRG1",
"cite_CD38",
"cite_CD127",
"cite_CD169")

In [ ]:
cite_gene<-
c(
"CD45",
"CD3",
"CD4",
"CD8",
"TCRab",
"TCRgd",
"TCRVa7.2",
"TCRVa24Ja18",
"CD45RA",
"CD45RO",
"PDL1",
"CD154",
"CD56",
"CD14",
"CD19",
"HLAABC",
"CD7",
"CD49f",
"CD25",
"TIGIT",
"CTLA4",
"CD31",
"CD44",
"PDPN",
"CD103",
"CD69",
"CD62L",
"CD161",
"LAG3",
"CD27",
"LAMP1",
"HLADR",
"NKG2D",
"CD57",
"KLRG1",
"CD38",
"CD127",
"CD169")

In [ ]:
seu@assays

In [ ]:
#DefaultAssay(seu) <- "CITE"
#seu<- NormalizeData(object = seu,
#                     normalization.method = "LogNormalize",
#                     scale.factor = 10000)

In [ ]:
for (n in cite_gene){
    print(FeaturePlot(object = seu,features = n,
                  cols = c("lightgrey", "blue"),
                      order = T,
                      slot = "data",
                      #min.cutoff = "q05", max.cutoff = "q95",
                      reduction = "umap",
                pt.size=1
                       ))   
      print(VlnPlot(object = seu, features = n,
                 #group.by = "sample",
                 #x.lab.rot=T, 
                 #size.x.use = 5,
                 pt.size = 0.01
                      )+NoLegend()
   )
   #print(RidgePlot(seu, 
   #          features = n))

    }

In [ ]:
DefaultAssay(seu)<-'RNA'

In [ ]:
DefaultAssay(seu)<-'RNA'
# Search for known marker genes in seu
leukos <- c("PTPRC") 
Tcells <- c("CD3G","CD3D","CD3E")
CD4<-c("CD4")
CD8 <- c("CD8A", "CD8B")
Naive<-c("LEF1","TCF7","LTB")
CM<-c("CCR7","SELL","KLF2","S1PR1")
RM<-c("CXCR6","CD69","ITGAE","RGS1")
Th1 <- c("CXCR3","TBX21", "IFNG", "TNF","CSF2")
Th2 <- c("GATA3", "IL4", "IL5", "IL13") 
Th17 <- c("CCR6","RORC","IL17A","IL17F", "IL23R") 
Tr1<-c("IL10","ITGA2","LAG3","HAVCR2")#,"Ahr","Irf4","Prdm1","Maf")
Tregs <- c("FOXP3", "IL2RA", "CTLA4","IKZF2","TIGIT")
Tfh<-c('IL21','POU2AF1','CXCR5','BCL6','ASCL2','CD200','ID3','ICOS','ICOSLG')
CTL<-c("PRF1", "GZMB","GZMK","GZMA","GZMH","GNLY")
NK <- c("NKG7","KLRC1","KLRD1","KLRF1","KLRB1","NCR1","NCAM1","FGFBP2","XCL1","XCL2")
Tgd<- c("TRDV2","TRGV9")
MAIT<-c("TRAV1-2")
Prolif<-c("STMN1","MKI67","TOP2A")

known_markers<-list(
leukos,
Tcells,
CD4,
CD8,
CTL,
NK,
Naive,
CM,
RM,
Th1,
Tr1,
Th2,
Th17,
MAIT,
Tgd,
Tregs,
Tfh,
Prolif
)
known_markers

marker_gene_list<-known_markers
length(unlist(marker_gene_list))
marker_gene_list_expressed <- intersect(unlist(marker_gene_list), rownames(GetAssayData(seu)))
length(marker_gene_list_expressed)
setdiff(unlist(marker_gene_list),marker_gene_list_expressed)

In [ ]:
for (n in marker_gene_list_expressed){
    print(FeaturePlot(object = seu,features = n,
                  cols = c("lightgrey", "blue"),
                      order = T,
                      slot = "data",
                      #min.cutoff = "q05", max.cutoff = "q95",
                      reduction = "umap",
                pt.size=1
                       ))   
      print(VlnPlot(object = seu, features = n,
                 #group.by = "sample",
                 #x.lab.rot=T, 
                 #size.x.use = 5,
                 pt.size = 0.01
                      )+NoLegend()
   )
          print(VlnPlot(object = seu, features = n,
                 #group.by = "sample",
                 #x.lab.rot=T, 
                 #size.x.use = 5,
                 pt.size = 0
                      )+NoLegend()
   )
   #print(RidgePlot(seu, 
   #          features = n))

    }

In [ ]:
DimPlot(object = seu, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
seu_T1<-subset(seu, subset = UMAP_1 > -8 & UMAP_2 > -8)
seu_T1

In [ ]:
DimPlot(object = seu_T1, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
#seu
seu_T<-subset(seu, idents=c("7","8","11","12","14"),invert=TRUE)
seu_T

In [ ]:
DimPlot(object = seu_T, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T) + theme(aspect.ratio=1)

In [ ]:
saveRDS(seu_T1, file = paste0("Integrated_objects/",samples_ID,"_T.rds"))

In [ ]:
saveRDS(seu, file = paste0("Integrated_objects/",samples_ID,"_all.rds"))